In [ ]:
import fitz
from PIL import Image
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from paddleocr import PaddleOCR
import signal
import sys
import time

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en', det_db_thresh=0.4, use_gpu = True)

pdf_folder_path = r"C:\Users\Ishwari Kumbhar\OneDrive\Desktop\Ornet\OCR extraction\127\271-300 (127)"
prefix_mapping_file = r"C:\Users\Ishwari Kumbhar\OneDrive\Desktop\Ornet\OCR extraction\Prefix.xlsx"
output_path = os.path.join(os.path.dirname(pdf_folder_path), "output_new_final_5 - Copy.xlsx")
emergency_save_path = os.path.join(os.path.dirname(pdf_folder_path), "emergency_save_5.xlsx")
sheet_name = "Igatpuri-127"  # Change sheet name here only
valid_prefixes = set()  # Initialize an empty set to prevent NameError
# Add these global variables at the top
interrupted = False
processed_files = set()


try:
    # Load the Excel file and check available sheet names
    xls = pd.ExcelFile(prefix_mapping_file)
    if sheet_name in xls.sheet_names:
        # Read prefix mapping sheet
        prefix_df = pd.read_excel(prefix_mapping_file, sheet_name=sheet_name, dtype={"FirstThreeLetters": str})

        # Convert 'cnt' column safely
        prefix_df["cnt"] = pd.to_numeric(prefix_df["cnt"], errors="coerce").fillna(0).astype(int)

        prefix_counts = prefix_df[["FirstThreeLetters", "cnt"]].sort_values(by="cnt", ascending=False)
        valid_prefixes = set(prefix_counts["FirstThreeLetters"].astype(str))  # Ensure it's a set of strings

        print("Valid Prefixes:", valid_prefixes)
    else:
        print(f"Sheet '{sheet_name}' not found. Skipping prefix processing.")
except Exception as e:
    print(f"Error loading prefix mapping file: {e}")

page_width, page_height = 595.0, 842.0
num_rows, num_cols = 10, 3
upper_lower_margin, side_margin, inner_spacing = 25, 15, 5

card_width = (page_width - 2 * side_margin - (num_cols - 1) * inner_spacing) / num_cols
card_height = (page_height - 2 * upper_lower_margin - (num_rows - 1) * inner_spacing) / num_rows

card_coordinates = [
    (int(side_margin * 2 + col * (card_width * 2 + inner_spacing * 2)),
     int(upper_lower_margin * 2 + row * (card_height * 2 + inner_spacing * 2)),
     int(card_width * 2), int(card_height * 2))
    for row in range(num_rows) for col in range(num_cols)
]

data_columns = {
    "File Name":[], "Serial Number": [], "Page Number": [], "Extracted Text": [],
    "EPIC Number": [], "Gender": [], "Age": [], "Name": [],
    "Relation": [], "Relation_Name": [], "Applicant_First_Name": [], "Applicant_Last_Name": [],
    "Relation_First_Name": [], "Relation_Last_Name": [], "AC_NO": [], "Part_No": [],
    "Booth Address": [], "House No": [], "Deleted_status": []
}

section_numbers_per_page = {}


def handle_interrupt(signum, frame):
    global interrupted
    print("\nInterrupt received. Saving progress before exiting...")
    interrupted = True
    save_progress()
    sys.exit(0)



def extract_text_with_paddleocr(image):
    """Extract text from an image using PaddleOCR."""
    try:
        # Convert the image to a numpy array
        image_np = np.array(image)

        # Debugging: Print type and shape of the numpy array
        print(f"Image type: {type(image_np)}")  # Should be <class 'numpy.ndarray'>
        print(f"Image shape: {image_np.shape}")  # Should be (height, width, channels)

        # Perform OCR on the image
        result = ocr.ocr(image_np, cls=True)

        # Handle cases where no text is detected
        if result is None:
            print("Warning: No text detected in the image.")
            
            return ""  # Return an empty string if no text is detected

        # Extract text from the OCR result
        text = ""
        for line in result:
            for word in line:
                if word and len(word) > 1:  # Ensure word has at least two elements
                    text += word[1][0] + " "  # Append the recognized text

        return text.strip()  # Return the cleaned text

    except Exception as e:
        print(f"Error during OCR processing: {e}")
        
        return ""  # Return an empty string in case of errors

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF using PaddleOCR."""
    doc = fitz.open(pdf_path)
    extracted_text = ""

    for page_num in range(len(doc)):
        page = doc[page_num]
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        extracted_text += extract_text_with_paddleocr(img) + "\n"

    return extracted_text

# Rest of the functions remain the same as in the original code...
def find_best_prefix(prefix):
    """Find the best valid prefix dynamically based on the first 2 letters."""
    prefix_2 = prefix[:2]  # Extract first 2 letters

    # Look for valid prefixes that start with the first 2 letters
    possible_matches = [p for p in valid_prefixes if p.startswith(prefix_2)]

    if possible_matches:
        #return max(possible_matches, key=len)  # Pick longest match
        #best_match = max(possible_matches, key=len)  # Pick the longest match
        #print(f"Checking best prefix for: {prefix} → Found: {possible_matches[0]}")  # ✅ Debug Step 2
        #return possible_matches[0]  # Pick the first (most frequent) match
        return sorted(possible_matches, key=lambda x: (len(x), x), reverse=True)[0]
    return None  # If no valid correction found, return original

def extract_epic_number(text):
    """Extract and correct EPIC numbers from text."""
    text = text.replace(" ", "")
    match = re.search(r"[a-zA-Z]{3,4}\d{7}", text)
    if match:
        epic_number = match.group(0)
        return correct_epic_number(epic_number)  # Apply correction
    return None

def correct_epic_number(epic_number):
    if not isinstance(epic_number, str):  # Ensure it's a string
        print(f"⚠️ Warning: epic_number is not a string: {epic_number}")
        return None

    epic_number = str(epic_number).strip()  # Ensure it's a string

    # Remove leading numbers before extracting prefix
    match = re.search(r"[a-zA-Z]{2,4}\d{7}$", epic_number)
    if not match:
        print(f"⚠️ Invalid EPIC format: {epic_number}")
        return epic_number  # Return as is if format is incorrect

    epic_number = match.group(0)  # Extract corrected EPIC format
    digits = epic_number[-7:]  # Extract numeric part
    prefix = epic_number[:-7].upper()  # Extract the alphabetic prefix

    prefix = prefix.upper()  # Convert to uppercase


    print(f"Extracted Prefix: {prefix}, Digits: {digits}")  # ✅ Debug Step 3

    # **Case 1: If 3-letter prefix is valid, return as is**
    if len(prefix) == 3:
       if prefix in valid_prefixes:
          print(f"✅ Valid Prefix: {prefix} → No Change")
          return epic_number
       else:
          corrected_prefix = find_best_prefix(prefix)  # 🔥 Find best match dynamically
          if corrected_prefix:
            print(f"🔄 Corrected {prefix} to {corrected_prefix}")
            return corrected_prefix + digits  # Apply correction
          return epic_number
    # **Case 2: If 2-letter prefix, find a matching 3-letter prefix**
    if len(prefix) == 2:
        corrected_prefix = find_best_prefix(prefix)
        if corrected_prefix in valid_prefixes:
            print(f"🔄 Corrected {prefix} to {corrected_prefix}")
            return corrected_prefix + digits
        return epic_number

    # **Case 3: If 4-letter prefix, try reducing to 3-letter**
    if len(prefix) == 4:
        for i in range(4):
            possible_prefix = prefix[:i] + prefix[i+1:]  # Remove one letter at a time
            if possible_prefix in valid_prefixes:
                print(f"🔄 Reduced {prefix} to {possible_prefix}")
                return possible_prefix + digits  # Use corrected prefix

    print(f"❌ No Change for {epic_number}")  # ✅ Debug Step 4
    return epic_number  # No correction found

    # If no correction found, return original
    return epic_number




import re

def extract_name(text):
    """
    Extracts the name from the text based on common patterns.
    Includes debug steps to track the input text and matched pattern.
    """
    # Debug: Print the input text
    print(f"Input Text: {text}")

    # Patterns to capture names after "Name", "Father's Name", and "Others"
    patterns = [
        # Updated Pattern 1: More specific to capture names after "Name :"
        r'Name\s*[:]?\s*([A-Za-z\s\.\-]+?)\s*(?=Father\'s Name|Husband\'s Name|Others|House Number|Age|Gender|$)'        
        # Keep the rest of the patterns unchanged
        r'(?<=Name[ :#?+\-].)(.?)(?=.Others)',  # Handles names after "Others" (with or without apostrophe)
        r'\bName\s*[^a-zA-Z]*(.*?)\s*(?:Mother|Father|Husband|Fathor|Other|\'s|Others)\b',  # Broad fallback
        r'\bName\s*[:]\s*([A-Za-z\s]+?)\s*(?:Mother|Father|Husband|Fathor|Other|\'s|Others|$)',  # Fallback for "Name :"
        r'(?:Name\s*[:#+]?\s*)([A-Za-z\s\.\-]+)(?=\s*(?:Father\'s Name:|Others:|Husband\'s Name:|House Number|‘Age|Available|Gender))',  # New pattern for variations
        r'Name\s*[:#?]?\s*([A-Za-z\s]+?)(?=\s*Others)'  # Regex pattern to match name between "Name" and "Others"
    ]
    
    # Try each pattern in the list
    for i, pattern in enumerate(patterns, start=1):
        match = re.search(pattern, text)
        if match:
            # Debug: Print the matched pattern and extracted name
            print(f"Pattern {i} Matched: {pattern}")
            print(f"Extracted Name: {match.group(1).strip()}")
            return match.group(1).strip()

    # Debug: If no pattern matches
    print("No Pattern Matched")
    return None

# Function to extract House No
def extract_house_no(text):
    # Match the text between "House Number" and "Age"
    match = re.search(r'House\s*Number\s*[:\-]?\s*(.*?)\s*(?:Age)', text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return None

# Function to convert string to camel case but retain spaces between words
def to_camel_case(name):
    return ' '.join([word.capitalize() for word in name.split()])

# Function to extract gender from the text
def extract_gender(text):
    if re.search(r'\b(female|[a-zA-Z]+female)\b', text, re.IGNORECASE):
        return "Female"
    elif re.search(r'\b(male|[a-zA-Z]+male)\b', text, re.IGNORECASE):
        return "Male"
    elif re.search(r'\bMalo\b', text, re.IGNORECASE):
        return "Male"
    elif re.search(r'\bFale\b', text, re.IGNORECASE):
        return "Female"
    return None

# Function to extract age from the text
def extract_age(text):
    match = re.search(r'age[^\d]*(\d{1,3})', text, re.IGNORECASE)
    if match:
        age = int(match.group(1))
        return age if age >= 18 else None
    two_digit_match = re.search(r'\b(\d{2})\b', text)
    return int(two_digit_match.group(1)) if two_digit_match and int(two_digit_match.group(1)) >= 18 else None

# Function to extract relation based on keywords
def extract_relation(text):
    """
    Extracts the relationship type (e.g., Husband, Father, Mother) from the text.
    """
    # Debug: Print the input text
    print(f"Input Text for Relation: {text}")

    # Updated regex to identify the relationship type
    if re.search(r'Husbands?\s*(?:Name|Narme|Narne)\s*:', text, re.IGNORECASE):
        print("Relation: Husband")
        return "Husband"
    elif re.search(r'Fathers?\s*(?:Name|Narme|Narne)\s*:', text, re.IGNORECASE):
        print("Relation: Father")
        return "Father"
    elif re.search(r'Mothers?\s*(?:Name|Narme|Narne)\s*:', text, re.IGNORECASE):
        print("Relation: Mother")
        return "Mother"
    elif re.search(r'Others?\s*(?:Name|Narme|Narne)\s*:', text, re.IGNORECASE):
        print("Relation: Others")
        return "Others"
    else:
        # Fallback: Look for simpler patterns like "Husband", "Father", etc.
        if re.search(r'\bhusband\b', text, re.IGNORECASE):
            print("Relation: Husband")
            return "Husband"
        elif re.search(r'\bfather\b', text, re.IGNORECASE):
            print("Relation: Father")
            return "Father"
        elif re.search(r'\bmother\b', text, re.IGNORECASE):
            print("Relation: Mother")
            return "Mother"
        elif re.search(r'\bothers\b', text, re.IGNORECASE):
            print("Relation: Others")
            return "Others"

    # Debug: Indicate no match
    print("No Relation Found")
    return None

# Function to extract relation name based on "Father's Name" or "Husband's Name"
def extract_relation_name(text):
    """
    Extracts the father's or husband's name from the text.
    The name is typically found after "Father's Name", "Husband's Name", or "Others".
    """
    # Pattern to capture the relation's name
    patterns = [
        r'(?:Father\'s Name|Husband\'s Name|Others)\s*[:#]?\s*([A-Za-z\s\.\-]+?)\s*(?:House Number|Age|Gender|$)',
        r'(?:Fathers Name|Husbands Name|Others)\s*[:#]?\s*([A-Za-z\s\.\-]+?)\s*(?:House Number|Age|Gender|$)',
        r'(?:Father|Husband|Others)\s*[:#]?\s*([A-Za-z\s\.\-]+?)\s*(?:House Number|Age|Gender|$)',
    ]

    # Try each pattern to extract the relation's name
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(1).strip()

    # If no match is found, return None
    return None
    
def extract_first_last_name(full_name):
    name_parts = full_name.strip().split()
    return name_parts[0] if name_parts else "", name_parts[-1] if len(name_parts) > 1 else ""


# Function to extract first and last name from a full name
def extract_section_number_and_name(text):
    # Multiple patterns to try matching
    patterns = [
        r"Section No and Name(?:[:\s]+)?(\d+)-([^\n;]+)(?:;)?",  # Original pattern
        r"Section\s*(?:No\.?|Number)?\s*(?:and)?\s*Name\s*:?\s*(\d+)\s*[-:]\s*([^\n;]+)",  # More flexible
        r"(?:Section|Sec\.?)\s*(\d+)\s*[-:]\s*([^\n;]+)",  # Even more flexible
        r"(\d+)\s*[-:]\s*([^\n;]+(?:Village|Area|Colony|Nagar|Road|Street|Lane|Mohalla))"  # Direct number-name pairs
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            section_number = match.group(1).strip()
            section_name = match.group(2).strip()
            print(f"Found section: {section_number} - {section_name}")  # Debug print
            return f"Section No and Name: {section_number}-{section_name}"
    
    print(f"No section match found in text: {text[:100]}...")  # Debug print
    return None

def extract_ac_no(text):
    # Try to match 3 digits, 2 digits, or 1 digit followed by a hyphen
    match = re.search(r"(\d{1,3})\s*-\s*", text)
    return match.group(1) if match else None

def extract_part_no(text):
    match = re.search(r"Part\s+No[.,]?\s*:\s*(\d+)", text)
    return match.group(1) if match else None

def extract_booth_address(text):
    match = re.search(r"Address of Polling Station\s*:\s*([^\n]+(?:\n[^\n]+)*)", text, re.DOTALL)
    return match.group(1).strip() if match else None

# Function to check and mark "Deleted_status" if extracted text starts with "s"
def check_deleted_status(extracted_text):
    return True if extracted_text.lower().startswith(('s ', 'r', 'r ', 'r2', 'r2 ', 'r3',  'r3 ','r4', 'r4 ', 'r5', 'r5 ' )) else False


# Iterate over all PDF files in the specified folder
def check_emergency_save():
    """Check for emergency save file and load processed files if it exists."""
    existing_data = None
    processed_pdfs = set()
    loaded_section_numbers = {}
    
    # Try main output file first
    if os.path.exists(output_path):
        try:
            print(f"\nFound existing output file at: {output_path}")
            existing_data = pd.read_excel(output_path)
            processed_pdfs.update(existing_data['File Name'].unique())
            print(f"Found {len(processed_pdfs)} previously processed PDFs in output file.")
            
            # Load ALL section information from existing file
            if all(col in existing_data.columns for col in ['Section Number', 'Section Name']):
                for _, row in existing_data.iterrows():
                    key = (row["File Name"], row["Page Number"])
                    if pd.notna(row['Section Number']) and pd.notna(row['Section Name']):
                        section_info = f"Section No and Name: {row['Section Number']}-{row['Section Name']}"
                        if key not in loaded_section_numbers:
                            loaded_section_numbers[key] = []
                        if section_info not in loaded_section_numbers[key]:
                            loaded_section_numbers[key].append(section_info)
            
            return processed_pdfs, existing_data, loaded_section_numbers
        except Exception as e:
            print(f"Error reading output file: {str(e)}")
    
    # Fallback to emergency save
    if os.path.exists(emergency_save_path):
        try:
            print(f"\nFound emergency save file at: {emergency_save_path}")
            existing_data = pd.read_excel(emergency_save_path)
            processed_pdfs.update(existing_data['File Name'].unique())
            print(f"Found {len(processed_pdfs)} previously processed PDFs in emergency save.")
            
            # Load ALL section information from emergency save
            if all(col in existing_data.columns for col in ['Section Number', 'Section Name']):
                for _, row in existing_data.iterrows():
                    key = (row["File Name"], row["Page Number"])
                    if pd.notna(row['Section Number']) and pd.notna(row['Section Name']):
                        section_info = f"Section No and Name: {row['Section Number']}-{row['Section Name']}"
                        if key not in loaded_section_numbers:
                            loaded_section_numbers[key] = []
                        if section_info not in loaded_section_numbers[key]:
                            loaded_section_numbers[key].append(section_info)
            
            return processed_pdfs, existing_data, loaded_section_numbers
        except Exception as e:
            print(f"Error reading emergency save file: {str(e)}")
    
    print("No existing data found. Starting fresh processing.")
    return set(), None, {}

def process_pdfs():
    global data_columns, section_numbers_per_page, interrupted, processed_files
    
    # Get all PDF files in the folder
    pdf_files = sorted([f for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')])
    total_files = len(pdf_files)
    
    # Check for existing output and get processed files and section numbers
    processed_files, existing_data, loaded_section_numbers = check_emergency_save()
    section_numbers_per_page.update(loaded_section_numbers)
    
    # If we have existing data, initialize data_columns with it
    if existing_data is not None:
        for column in existing_data.columns:
            if column in data_columns:
                data_columns[column] = existing_data[column].tolist()
    
    pdf_files_to_process = [f for f in pdf_files if f not in processed_files]
    
    print(f"Found {total_files} PDFs in total. {len(pdf_files_to_process)} remaining to process.")
    
    try:
        for pdf_file in tqdm(pdf_files_to_process, desc="Processing PDFs", unit="file"):
            if interrupted:
                break
                
            try:
                pdf_path = os.path.join(pdf_folder_path, pdf_file)
                temp_data = {key: [] for key in data_columns}
                temp_section_numbers = {}
                
                with fitz.open(pdf_path) as pdf_document:
                    total_pages = pdf_document.page_count

                    # Process first two pages for header info
                    page_1 = pdf_document[0]
                    pix_1 = page_1.get_pixmap(matrix=fitz.Matrix(2, 2))
                    img_1 = Image.frombytes("RGB", [pix_1.width, pix_1.height], pix_1.samples).convert("L")
                    page_1_text = extract_text_with_paddleocr(img_1)
                    cleaned_page_1_text = re.sub(r'\s+', ' ', page_1_text).strip()

                    page_2 = pdf_document[1]
                    pix_2 = page_2.get_pixmap(matrix=fitz.Matrix(2, 2))
                    img_2 = Image.frombytes("RGB", [pix_2.width, pix_2.height], pix_2.samples).convert("L")
                    page_2_text = extract_text_with_paddleocr(img_2)
                    cleaned_page_2_text = re.sub(r'\s+', ' ', page_2_text).strip()

                    ac_no = extract_ac_no(cleaned_page_2_text)
                    part_no = extract_part_no(cleaned_page_1_text) or extract_part_no(cleaned_page_2_text)
                    booth_address = extract_booth_address(cleaned_page_2_text) or extract_booth_address(cleaned_page_1_text)

                    page_1_data = {
                        "AC_NO": ac_no,
                        "Part_No": part_no,
                        "Booth Address": booth_address
                    }

                    serial_counter = 1

                    # Process each page in the PDF
                    for page_num in range(1, total_pages):
                        if interrupted:
                            break
                            
                        try:
                            page = pdf_document[page_num]
                            pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
                            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples).convert("L")
                            page_text = extract_text_with_paddleocr(img).strip()

                            section_no_and_name = extract_section_number_and_name(page_text)
                            if section_no_and_name:
                                key = (pdf_file, page_num + 1)
                                if key in temp_section_numbers:
                                    temp_section_numbers[key].append(str(section_no_and_name))
                                else:
                                    temp_section_numbers[key] = [str(section_no_and_name)]

                            # Process each card on the page
                            for (x, y, w, h) in card_coordinates:
                                if interrupted:
                                    break
                                    
                                card_image = img.crop((x, y, x + w, y + h))
                                extracted_text = extract_text_with_paddleocr(card_image).strip()
                                extracted_text = extracted_text.replace("Photo", "").strip()
                                extracted_text = extracted_text.replace("Available","").strip()

                                if extracted_text:
                                    temp_data["Serial Number"].append(serial_counter)
                                    temp_data["Page Number"].append(page_num + 1)
                                    temp_data["Extracted Text"].append(extracted_text)
                                    temp_data["AC_NO"].append(page_1_data["AC_NO"])
                                    temp_data["Part_No"].append(page_1_data["Part_No"])
                                    temp_data["Booth Address"].append(page_1_data["Booth Address"])
                                    temp_data["Deleted_status"].append(check_deleted_status(extracted_text))
                                    temp_data["House No"].append(extract_house_no(extracted_text))
                                    temp_data["File Name"].append(pdf_file)

                                    serial_counter += 1

                                    epic_number = extract_epic_number(extracted_text)
                                    if not epic_number:
                                        extracted_text = extract_text_with_paddleocr(card_image).strip()
                                        epic_number = extract_epic_number(extracted_text)
                                    temp_data["EPIC Number"].append(epic_number)

                                    temp_data["Gender"].append(extract_gender(extracted_text))
                                    temp_data["Age"].append(extract_age(extracted_text))
                                    
                                    name = extract_name(extracted_text)
                                    temp_data["Name"].append(name)
                                    
                                    relation = extract_relation(extracted_text)
                                    temp_data["Relation"].append(relation)
                                    
                                    relation_name = extract_relation_name(extracted_text) or (' '.join(name.split()[1:]) if name else None)
                                    temp_data["Relation_Name"].append(relation_name)

                                    applicant_first, applicant_last = extract_first_last_name(name or "")
                                    temp_data["Applicant_First_Name"].append(applicant_first)
                                    temp_data["Applicant_Last_Name"].append(applicant_last)
                                    
                                    relation_first, relation_last = extract_first_last_name(relation_name or "")
                                    temp_data["Relation_First_Name"].append(relation_first)
                                    temp_data["Relation_Last_Name"].append(relation_last)

                        except Exception as e:
                            print(f"Error processing page {page_num} in {pdf_file}: {str(e)}")
                            continue

                # Only merge data if we didn't get interrupted
                if not interrupted:
                    for key in data_columns:
                        data_columns[key].extend(temp_data[key])
                    section_numbers_per_page.update(temp_section_numbers)
                    processed_files.add(pdf_file)
                    
                    # Save after each successful PDF processing
                    save_progress()
                
            except Exception as e:
                print(f"Error processing file {pdf_file}: {str(e)}")
                print(traceback.format_exc())
                continue

    finally:
        # Final save attempt in case of normal completion or interruption
        if not interrupted:
            print("\nProcessing completed normally.")
        save_progress()

# Rest of the post-processing functions remain the same...
def map_section_number(row):
    key = (row["File Name"], row["Page Number"])
    
    # First try to use existing section info if available
    if 'Section Number' in row and 'Section Name' in row:
        if pd.notna(row['Section Number']) and pd.notna(row['Section Name']):
            return f"Section No and Name: {row['Section Number']}-{row['Section Name']}"
    
    # Then check loaded section numbers
    sections = section_numbers_per_page.get(key, [])
    if sections:
        return ", ".join(filter(None, sections))
    
    # Finally return None if nothing found
    return None



# function to remove the rows containing page number 2
def remove_page_number_rows(df):

    if 'Page Number' in df.columns:
        # Filter out rows where 'Page Number' is 1 or 2
        df = df[~df['Page Number'].isin([1, 2])]
    else:
        print("Column 'Page Number' not found in the DataFrame.")
    return df

def extract_serial_number(text):
    if not isinstance(text, str):
        return None
    match = re.match(r"^\d+", text.strip())  # Match numbers at the beginning of the text
    return int(match.group()) if match else None

# Map serial numbers if not matching within the given range
# def map_serial_numbers(df):
#     for i in range(0, len(df), 2):  # Process the DataFrame in chunks of 2 rows
#         chunk = df.iloc[i:i+2]  # Ensure slicing by position

#         # Get valid serial numbers from the "Extracted Serial Number" column
#         valid_serials = chunk["Extracted Serial Number"].dropna()

#         if not valid_serials.empty:
#             # If there are valid serial numbers, get the first valid serial number
#             first_valid_serial = valid_serials.iloc[0]
#         else:
#             # If no valid serial number is found, create a fallback serial number
#             # This happens when "Extracted Serial Number" is empty (NaN)
#             if i == 0 or pd.isna(df.loc[i-1, "Serial Number"]):
#                 first_valid_serial = 1  # Start from 1 or some other default number
#             else:
#                 first_valid_serial = df.loc[i-1, "Serial Number"] + 1  # Increment from the previous serial number

#         # Ensure first_valid_serial is an integer
#         first_valid_serial = int(first_valid_serial)

#         # Update the "Serial Number" column with sequential values
#         serial_numbers = list(range(first_valid_serial, first_valid_serial + len(chunk)))

#         # Check if the "Extracted Serial Number" is NaN, and increment the serial number from the previous row
#         for j in range(len(chunk)):
#             if pd.isna(chunk.iloc[j]["Extracted Serial Number"]):
#                 # Increment the serial number from the previous row
#                 serial_numbers[j] = df.loc[i + j - 1, "Serial Number"] + 1

#         # Apply the updated serial numbers
#         df.iloc[i:i+2, df.columns.get_loc("Serial Number")] = serial_numbers

#     return df

# Function to map serial numbers if they don't match
def map_serial_numbers(df):
    # Group the DataFrame by file name
    for file_name, file_group in df.groupby("File Name"):
        current_serial = 1  # Reset for each file
        for i in range(0, len(file_group)):  # Process row by row
            row = file_group.iloc[i]
            extracted_serial = row["Extracted Serial Number"]

            # If extracted_serial is valid, use it
            if pd.notna(extracted_serial) and str(extracted_serial).isdigit():
                df.at[row.name, "Serial Number"] = int(extracted_serial)
                current_serial = int(extracted_serial) + 1  # Update current_serial
            else:
                # If extracted_serial is invalid, use current_serial
                df.at[row.name, "Serial Number"] = current_serial
                current_serial += 1  # Increment current_serial
    return df

# Update serial numbers based on the 'Page Number' column
#def update_serial_numbers(df):
 #   if df.empty:
  #      raise ValueError("DataFrame is empty. Ensure it contains data before processing.")

   # if "Page Number" not in df.columns:
    #    raise KeyError("The column 'Page Number' is missing in the DataFrame.")

  #  df["Serial Number"] = None
   # current_serial = 1
    #updated_serials = []
 #   previous_page = None

  #  for index, row in df.iterrows():
   #     current_page = row["Page Number"]
    #    if previous_page is None or current_page < previous_page:
     #       current_serial = 1
      #  updated_serials.append(current_serial)
       # current_serial += 1
        #previous_page = current_page

    #df["Serial Number"] = updated_serials
    #return df

def clean_booth_address(df):
    # Check if the "Booth Address" column exists in the dataframe
    if "Booth Address" in df.columns:
        # Modify the "Booth Address" column
        df["Booth Address"] = df["Booth Address"].apply(
            lambda x: re.sub(r"(\d{6})(.*)", r"\1", x) if isinstance(x, str) else x
        )

        # Remove the starting text "Stations in this part :"
        df["Booth Address"] = df["Booth Address"].apply(
            lambda x: x.replace("Stations in this part :", "").strip() if isinstance(x, str) and x.startswith("Stations in this part :") else x
        )

        # Remove the "NUMBER OF ELECTORS" section and everything after it if no pincode is present
        def remove_electors_info(address):
            if isinstance(address, str):
                # Check if address has no pincode and contains "NUMBER OF ELECTORS"
                if not re.search(r"\d{6}", address):
                    # Adjust the regex to remove "NUMBER OF ELECTORS" and everything after it
                    address = re.sub(r"(\d+\.\s*,?\s*NUMBER OF ELECTORS.*|4\.\s*,?\s*NUMBER OF ELECTORS.*)", "", address).strip()
            return address

        df["Booth Address"] = df["Booth Address"].apply(remove_electors_info)

        # Remove text after the 4th comma (including the 4th comma) if no pincode is present, otherwise up to the 3rd comma
        def remove_after_4th_comma_if_no_pincode(address):
            if isinstance(address, str) and not re.search(r"\d{6}", address):  # No pincode found
                parts = address.split(",")

                # Check the number of commas and trim accordingly
                if len(parts) > 4:
                    return ",".join(parts[:4])  # If 4 commas, return first 4 parts
                elif len(parts) > 3:
                    return ",".join(parts[:3])  # If 3 commas, return first 3 parts
                elif len(parts) > 2:
                    return ",".join(parts[:2])  # If 2 commas, return first 2 parts
                elif len(parts) > 1:
                    return ",".join(parts[:1])  # If 1 comma, return first part
            return address

        df["Booth Address"] = df["Booth Address"].apply(remove_after_4th_comma_if_no_pincode)

    else:
        print("The 'Booth Address' column is not found in the DataFrame.")

    return df

def clean_section_name(section_name):
    """Dynamically extracts and cleans the section name."""
    if not isinstance(section_name, str) or not section_name.strip():
        return section_name  # Return as-is if empty or not a string

    # Remove EPIC numbers (format: 3 uppercase letters followed by 7 digits)
    section_name = re.sub(r'\b[A-Z]{3}\d{7}\b', '', section_name)

    # Remove numeric serial numbers (standalone or before keywords like "Name", "Photo", "House Number", etc.)
    section_name = re.sub(r'\b\d+\b', '', section_name)

    # Remove common unwanted keywords (like "Name :", "House Number", "Photo", "Age", "Gender", etc.)
    section_name = re.split(r"(?i)\b(Name|House Number|Photo|Age|Gender|Available|Date of Publication|Total Pages|Page)\b", section_name)[0]

    # Remove extra spaces and strip trailing commas
    section_name = section_name.strip().rstrip(",")

    return section_name


def split_section_columns(df):
    # Define the regex pattern to split "Serial Number and Name"
    pattern = r"Section No and Name\s*[:]?\s*(\d+)-([^\n]+)"

    # Create two new columns by applying the regex pattern to the 'Serial Number and Name' column
    df[['Section Number', 'Section Name']] = df['Section Number and Name'].str.extract(pattern)

    # Drop the original 'Serial Number and Name' column
    df.drop(columns=['Section Number and Name'], inplace=True)

    return df


def remove_house_number_text(df):

    if 'Name' not in df.columns:
        raise ValueError("The DataFrame does not contain a column named 'Name'")

    # Remove 'house number' text from the 'Name' column while handling NaN values
    df['Name'] = df['Name'].fillna('').str.replace('house number', '', case=False).str.strip()

    # Restore NaN values (if any were originally present)
    df.loc[df['Name'] == '', 'Name'] = pd.NA

    return df

def filter_age_column(df):
    # Check if 'Age' column exists in the DataFrame
    if 'Age' not in df.columns:
        raise ValueError("The DataFrame does not contain a column named 'Age'")

    # Replace values outside the range 18-100 with NaN
    df['Age'] = df['Age'].apply(lambda x: x if pd.notna(x) and 18 <= x <= 100 else np.nan)

    return df


def clean_name_and_relation_name(df):
    # Define words to remove from 'Name' and 'Relation_Name' columns
    unwanted_words = ['age', 'house', 'number', 'name', 'husband', 'mother', 'father']

    # Remove unwanted words from 'Name' and 'Relation_Name' columns
    for word in unwanted_words:
        df['Name'] = df['Name'].str.replace(rf'\b{word}\b', '', case=False, regex=True)
        df['Relation_Name'] = df['Relation_Name'].str.replace(rf'\b{word}\b', '', case=False, regex=True)
        df['Applicant_First_Name'] = df['Applicant_First_Name'].str.replace(rf'\b{word}\b', '', case=False, regex=True)
        df['Applicant_Last_Name'] = df['Applicant_Last_Name'].str.replace(rf'\b{word}\b', '', case=False, regex=True)
        df['Relation_First_Name'] = df['Relation_First_Name'].str.replace(rf'\b{word}\b', '', case=False, regex=True)
        df['Relation_Last_Name'] = df['Relation_Last_Name'].str.replace(rf'\b{word}\b', '', case=False, regex=True)

    return df


def save_progress():
    """Save current progress to the output file, handling errors gracefully."""
    try:
        print("\nSaving progress...")
        current_df = pd.DataFrame(data_columns)
        
        if not current_df.empty:
            current_df = post_process_excel(current_df)
            
            # Create output directory if it doesn't exist
            output_dir = os.path.dirname(output_path)
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            try:
                final_df = current_df
                
                # Try to merge with existing data if it exists
                if os.path.exists(output_path):
                    existing_df = pd.read_excel(output_path)
                    
                    # Only keep rows from existing data that aren't in current processing
                    existing_df = existing_df[~existing_df['File Name'].isin(current_df['File Name'].unique())]
                    
                    # Merge while preserving section info
                    final_df = pd.concat([existing_df, current_df], ignore_index=True)
                    
                    # Ensure section columns exist
                    if 'Section Number' not in final_df.columns:
                        final_df['Section Number'] = None
                    if 'Section Name' not in final_df.columns:
                        final_df['Section Name'] = None
                
                # Save to temporary file first
                temp_path = output_path + '.temp.xlsx'
                final_df.to_excel(temp_path, index=False)
                
                # Replace the original file
                if os.path.exists(output_path):
                    os.replace(temp_path, output_path)
                else:
                    os.rename(temp_path, output_path)
                print(f"Successfully saved to {output_path}")
                
                # Also save an emergency backup
                final_df.to_excel(emergency_save_path, index=False)
                
            except Exception as e:
                print(f"Error saving to main output: {str(e)}")
                # Emergency save with just current data
                current_df.to_excel(emergency_save_path, index=False)
                print(f"Saved emergency backup to {emergency_save_path}")
                
    except Exception as e:
        print(f"Critical error during save process: {str(e)}")

def post_process_excel(df):
    # Initial processing
    df = remove_page_number_rows(df)
    df["Extracted Serial Number"] = df["Extracted Text"].apply(extract_serial_number)
    df = map_serial_numbers(df)
    
    # Handle section information carefully
    if 'Section Number and Name' not in df.columns:
        df["Section Number and Name"] = df.apply(map_section_number, axis=1)
    
    # Filter empty rows
    df = df[~((df["EPIC Number"].isnull() | (df["EPIC Number"] == "")) &
              (df["Name"].isnull() | (df["Name"] == "")))]
    
    # Clean data
    df = clean_booth_address(df)
    df = df.drop(columns=["Extracted Serial Number"], errors='ignore')
    
    # Split section columns if needed
    if 'Section Number and Name' in df.columns and 'Section Number' not in df.columns:
        df = split_section_columns(df)
    
    # Clean section names if column exists
    if 'Section Name' in df.columns:
        df['Section Name'] = df['Section Name'].apply(clean_section_name)
    
    # Other processing steps
    #df = modify_epic_number(df)
    df = remove_house_number_text(df)
    df = filter_age_column(df)
    df = clean_name_and_relation_name(df)
    #df = modify_epic_number_10_digits(df)
    
    # Ensure all columns are present
    for col in ['Section Number', 'Section Name']:
        if col not in df.columns:
            df[col] = None
    
    return df
    # converting the dataframe to capital lettersS
    # df = df.applymap(lambda x: ", ".join(map(str, x)) if isinstance(x, (list, np.ndarray)) else str(x))
    # df["EPIC Number"] = df["EPIC Number"].apply(correct_epic_number)

# With this enhanced main execution block:
if __name__ == "__main__":
    # Register signal handlers
    signal.signal(signal.SIGINT, handle_interrupt)
    signal.signal(signal.SIGTERM, handle_interrupt)
    
    print("Starting PDF processing. Press Ctrl+C to save and exit.")
    
    # Check for emergency save and get processed files
    processed_files = check_emergency_save()
    
    # Initialize data structures
    data_columns = {key: [] for key in data_columns}  # Reset data columns
    section_numbers_per_page = {}  # Reset section numbers
    
    # Process all PDFs
    process_pdfs()
    
    # Post-process and save final output
    df = pd.DataFrame(data_columns)
    if not df.empty:
        df = post_process_excel(df)
        
        # Final save with checks
        temp_path = output_path + '.temp.xlsx'
        df.to_excel(temp_path, index=False)
        
        if os.path.exists(output_path):
            os.replace(temp_path, output_path)
        else:
            os.rename(temp_path, output_path)d
            
        print(f"\nProcessing complete. Final data saved to {output_path}")
        print(f"Total files processed: {len(processed_files)}")
        
        # If processing completed successfully, remove emergency save
        if os.path.exists(emergency_save_path):
            try:
                os.remove(emergency_save_path)
                print("Emergency save file removed after successful processing.")
            except Exception as e:
                print(f"Warning: Could not remove emergency save file: {str(e)}")
    else:
        print("No data was processed.")


[2025/05/02 21:22:05] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Ishwari Kumbhar/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.4, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Ishwari Kumbhar/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 3

Processing PDFs:   0%|          | 0/5 [00:00<?, ?file/s]

Image type: <class 'numpy.ndarray'>
Image shape: (1684, 1190)
[2025/05/02 21:22:13] ppocr DEBUG: dt_boxes num : 65, elapsed : 0.6015088558197021
[2025/05/02 21:22:14] ppocr DEBUG: cls num  : 65, elapsed : 0.22126126289367676
[2025/05/02 21:22:21] ppocr DEBUG: rec_res num  : 65, elapsed : 7.385350942611694
Image type: <class 'numpy.ndarray'>
Image shape: (1684, 1190)
[2025/05/02 21:22:22] ppocr DEBUG: dt_boxes num : 20, elapsed : 0.18684029579162598
[2025/05/02 21:22:22] ppocr DEBUG: cls num  : 20, elapsed : 0.2837073802947998
[2025/05/02 21:22:23] ppocr DEBUG: rec_res num  : 20, elapsed : 0.6427412033081055
Image type: <class 'numpy.ndarray'>
Image shape: (1684, 1190)
[2025/05/02 21:22:23] ppocr DEBUG: dt_boxes num : 20, elapsed : 0.04912710189819336
[2025/05/02 21:22:23] ppocr DEBUG: cls num  : 20, elapsed : 0.02431774139404297
[2025/05/02 21:22:23] ppocr DEBUG: rec_res num  : 20, elapsed : 0.37516093254089355
No section match found in text: Assembly Constituency No and Name:127-IGATP

Processing PDFs:  20%|██        | 1/5 [01:37<06:31, 97.98s/file]

Image type: <class 'numpy.ndarray'>
Image shape: (1684, 1190)
[2025/05/02 21:23:51] ppocr DEBUG: dt_boxes num : 72, elapsed : 0.09038162231445312
[2025/05/02 21:23:51] ppocr DEBUG: cls num  : 72, elapsed : 0.07144522666931152
[2025/05/02 21:23:54] ppocr DEBUG: rec_res num  : 72, elapsed : 2.8888373374938965
Image type: <class 'numpy.ndarray'>
Image shape: (1684, 1190)
[2025/05/02 21:23:54] ppocr DEBUG: dt_boxes num : 18, elapsed : 0.07641196250915527
[2025/05/02 21:23:54] ppocr DEBUG: cls num  : 18, elapsed : 0.09520268440246582
[2025/05/02 21:23:55] ppocr DEBUG: rec_res num  : 18, elapsed : 0.4999392032623291
Image type: <class 'numpy.ndarray'>
Image shape: (1684, 1190)
[2025/05/02 21:23:55] ppocr DEBUG: dt_boxes num : 18, elapsed : 0.04059028625488281
[2025/05/02 21:23:55] ppocr DEBUG: cls num  : 18, elapsed : 0.027952194213867188
[2025/05/02 21:23:55] ppocr DEBUG: rec_res num  : 18, elapsed : 0.4720897674560547
No section match found in text: Assembly Constituency No and Name:127-IG

Processing PDFs:  40%|████      | 2/5 [04:38<07:19, 146.35s/file]

Image type: <class 'numpy.ndarray'>
Image shape: (1684, 1190)
[2025/05/02 21:26:51] ppocr DEBUG: dt_boxes num : 69, elapsed : 0.08512687683105469
[2025/05/02 21:26:51] ppocr DEBUG: cls num  : 69, elapsed : 0.09241461753845215
[2025/05/02 21:26:52] ppocr DEBUG: rec_res num  : 69, elapsed : 0.8851926326751709
Image type: <class 'numpy.ndarray'>
Image shape: (1684, 1190)
[2025/05/02 21:26:52] ppocr DEBUG: dt_boxes num : 14, elapsed : 0.04958748817443848
[2025/05/02 21:26:52] ppocr DEBUG: cls num  : 14, elapsed : 0.015904903411865234
[2025/05/02 21:26:52] ppocr DEBUG: rec_res num  : 14, elapsed : 0.11321067810058594
Image type: <class 'numpy.ndarray'>
Image shape: (1684, 1190)
[2025/05/02 21:26:52] ppocr DEBUG: dt_boxes num : 14, elapsed : 0.038442134857177734
[2025/05/02 21:26:52] ppocr DEBUG: cls num  : 14, elapsed : 0.026998519897460938
[2025/05/02 21:26:53] ppocr DEBUG: rec_res num  : 14, elapsed : 0.08705019950866699
No section match found in text: Assembly Constituency No and Name:12